In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# Vectrix Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Creating a new project



In [1]:
from vectrix.db import DB
db = DB()
print(db.get_collection_metdata("Vectrix"))

[{'url': 'https://www.vectrix.ai/', 'uuid': 'd74613d5-4cb4-4e74-b886-93d5db7b1308', 'title': 'Vectrix - SLM Training & AI Solutions', 'language': None, 'source_type': 'webpage', 'source_format': 'html'}, {'url': 'https://www.vectrix.ai/blog-post/are-llm-benchmarks-and-leaderboards-just-marketing-tools', 'uuid': 'd4b0a47c-9503-42dc-8b86-7e9560518eb7', 'title': 'Are LLM Benchmarks and Leaderboards Just Marketing Tools?', 'language': None, 'source_type': 'webpage', 'source_format': 'html'}, {'url': 'https://www.vectrix.ai/offerings', 'uuid': '02c40731-7c71-44c1-a621-b5dedee02aa1', 'title': 'Vectrix Offerings - SLM Training & AI Solutions', 'language': None, 'source_type': 'webpage', 'source_format': 'html'}, {'url': 'https://www.vectrix.ai/blog-post/google-deepminds-searchless-chess-engine---part-1', 'uuid': 'abab98d0-dadb-41dc-adce-b80c29c3c625', 'title': 'Google DeepMind’s Searchless Chess Engine - Part 1', 'language': None, 'source_type': 'webpage', 'source_format': 'html'}, {'url': 'h

In [1]:
from vectrix.db import DB

db = DB()
db.create_project("Vectrix", description="Demo project for Vectrix")

TypeError: DB.__init__() missing 1 required positional argument: 'project_name'

In [ ]:
db.remove_project("Vectrix")

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**<>

This cell demonstrates the process of crawling a website, chunking the extracted content, and performing Named Entity Recognition (NER) using the Vectrix library.

#### Steps:

1. **Web Crawling**: 
   - Uses `vectrix.Crawler` to extract pages from "https://vectrix.ai"
   - Limits the crawl to a maximum of 20 pages

2. **Content Chunking**:
   - Utilizes `vectrix.Webchunker` to break down the extracted content
   - Chunks are created with a size of 500 characters and an overlap of 50 characters

3. **Named Entity Recognition**:
   - Employs `vectrix.Extract` with the 'Replicate' model
   - Uses the 'meta/meta-llama-3-70b-instruct' model for entity extraction

This example showcases a typical workflow for web data extraction and processing using the Vectrix library.

In [3]:
from vectrix.importers import WebScraper

scraper = WebScraper("https://www.vectrix.ai/")
all_links = scraper.get_all_links()

2024-08-18 18:23:38,203 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,203 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,203 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,511 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,511 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,511 - trafilatura.downloads - ERROR - not a 200 response: 404 for URL https://www.vectrix.ai/robots.txt
2024-08-18 18:23:38,555 - courlan.urlstore - WARNING - Discarding URL: /blog-post/google-deepminds-searchless-chess-engine---part-1
2024-08-18 18:23:38,555 - courlan.urlstore - WARNING - Discarding URL: /blog-post/google-deepminds-searchless-chess-engine--

In [4]:
web_pages = scraper.get_all_pages(all_links)
len(web_pages)

21

In [5]:
from vectrix.importers import chunk_content
chunked_webpages = chunk_content(web_pages)


print(f"Before chunking we had {len(web_pages)} and after chunking {len(chunked_webpages)}")

Before chunking we had 21 and after chunking 21


## Adding the data to a VectorStore

In [6]:
db.add_documents(chunked_webpages, "Vectrix")

2024-08-18 18:25:01,703 - langchain_cohere.utils - WARNING - Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised InternalServerError: status_code: 500, body: {'message': 'internal server error, this has been reported to our developers. id 34560669-33e7-4d6c-a360-64cc3482bff5'}.
2024-08-18 18:25:01,703 - langchain_cohere.utils - WARNING - Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised InternalServerError: status_code: 500, body: {'message': 'internal server error, this has been reported to our developers. id 34560669-33e7-4d6c-a360-64cc3482bff5'}.
2024-08-18 18:25:01,703 - langchain_cohere.utils - WARNING - Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised InternalServerError: status_code: 500, body: {'message': 'internal server error, this has been reported to 

In [1]:
print(db.similarity_search("Vectrix", "When was Vectrix founded?", 1))

NameError: name 'db' is not defined

### From OneDrive ☁
We can also connect with a OneDrive folder and download the information from there. After this process we can add the information to our Vector store in the same way. This way we can extract various kinds of documents like PDF, World, PowerPoint ...

Downloads files from OneDrive, processes them with Unstructured, and stores in Weaviate.

Steps:
1. Connect to OneDrive using Azure credentials
2. List and download all files
3. Process files with Unstructured importer
4. Add processed documents to Weaviate vector store
5. Close OneDrive connection

Requirements:
- vectrix library (OneDrive connector, Unstructured importer)
- Weaviate client
- Azure credentials as environment variables (AZURE_CLIENT_ID, AZURE_CLIENT_SECRET)

Note: Close OneDrive connection before uploading to vector store.

In [ ]:
from vectrix.connectors.onedrive import OneDrive
from vectrix.importers.unstructured import Unstructured

drive = OneDrive(azure_client_id = os.environ.get('AZURE_CLIENT_ID'), azure_client_secret = os.environ.get('AZURE_CLIENT_SECRET'))

drive.list_files()

In [ ]:
# Download all files in the drive, returns a list of downloaded files
downloaded_files = drive.download_files()
print(downloaded_files)

# Process teh documents using unstructured (we can't load RAW PDF files into a Vector store)
importer = Unstructured()
documents = importer.process_files(downloaded_files)

# Add the documents to the Vector store
weaviate = Weaviate()
weaviate.set_colleciton('Vectrix')
weaviate.add_data(documents)

# Close the drive, do this BEFORE you want to upload the files to a vector store
drive.close()